In [14]:
# -*- coding: utf-8 -*-
# Recurrent Neural Network

import copy
import os
import numpy as np
import json
import math
import random
import time

# Parameters Preset
alpha = 0.05
lamda = 0.001
mask = 10

# Data load in
data = []
training_set = []
raw_data = open("user_cart.json",'r')
unum = raw_data.read().count('\n')
raw_data.seek(0)
unum = 0
inum = 0
lines = raw_data.readlines()
for li in lines:
    record = json.loads(li)
    if(len(record)>10):
        thisline = []
        for i in range(len(record)):
            thisline.append(int(record[i]))
            if(int(record[i])>inum):
                inum = int(record[i])
        data.append(thisline)
        training_set.append(thisline[0:int(len(record)*0.8)])
        unum = unum + 1
print unum, inum
# Data understanding
# user_size 703(737) item_size 8533

702 8532


In [16]:
# Represent -- 10D features
np.random.seed(0)
I = np.random.randn(inum+1, mask) * 0.5
H = np.zeros((unum+1, mask))
U = np.random.randn(mask, mask) * 0.5
R = np.random.randn(mask, mask) * 0.5

# The random J choose
opt_set = []

def ResetOpt():
    global opt_set
    opt_set = []
    count = 0
    for user in xrange(unum):
        new_rand = []
        for item in xrange(len(training_set[user])):
            j = random.randint(0,8532)
            while(j == training_set[user][item]):
                j = random.randint(0, 8532)
            new_rand.append(j)
        opt_set.append(new_rand)
    #print len(opt_set)

In [17]:
def sigmond(x):
    return .5 *(1 + np.tanh(.5 * x))

In [ ]:
def train():
    global I, U, R, H
    #ResetOpt()
    for user in xrange(unum):
        isum = len(training_set[user])
        
        H3 = np.zeros((1, 10))
        H3 = H3[0]
        # The first item
        item = 0
        #feed-forward
        i = training_set[user][item] -1 
        I2 = I[i]
        A2 = np.dot(H3, R) + np.dot(I2, U)
        H2 = np.zeros((1, mask))
        H2 = H2[0]
        for k in xrange(mask):
            H2[k] = sigmond(A2[k])
        
        # The second item
        item = 1
        #feed-forward
        i = training_set[user][item] -1
        j = opt_set[user][item] -1
        I1 = I[i]
        J1 = I[j]
        A1 = np.dot(H2, R) + np.dot(I1, U)
        H1 = np.zeros((1, 10))
        H1 = H1[0]
        for k in xrange(mask):
            H1[k] = sigmond(A1[k])
        X = np.dot(H2, I1.T) - np.dot(H2, J1.T)
        dI1 = H2
        dJ1 = -H2
        I[i] += alpha *((1 - sigmond(X))* dI1 - lamda * I1)
        I[j] += alpha *((1 - sigmond(X))* dJ1 - lamda * J1)
        I1 = I[i]
        J1 = I[j]
        # back-propagation for once
        dXH2 = I1 - J1
        dHA2 = np.zeros((1, 10))
        dHA2 = dHA2[0]
        for k in xrange(mask):
            dHA2[k] = H2[k] *(1 - H2[k])
        dU = np.zeros((10, 10))
        dR = np.zeros((10, 10))
        Ut = U
        Rt = R
        U += alpha *((1 - sigmond(X))* dU - lamda * Ut)
        R += alpha *((1 - sigmond(X))* dR - lamda * Rt)
        
        #the third item
        item = 2
        #feed-forward
        i = training_set[user][item]-1
        j = opt_set[user][item]-1
        Inow = I[i]
        Jnow = I[j]
        X = np.dot(H1, Inow.T) - np.dot(H1, Jnow.T)
        dXInow = H1
        dXJnow = -H1
        I[i] += alpha *((1 - sigmond(X))* dXInow - lamda * Inow)
        I[j] += alpha *((1 - sigmond(X))* dXJnow - lamda * Jnow)
        Inow = I[i]
        Jnow = I[j]
        #back-propagation --first
        dXH1 = Inow - Jnow
        dHA1 = np.zeros((1, 10))
        for k in xrange(mask):
            dHA1[0][k] = H1[k] *(1 - H1[k])
        dU = np.zeros((10, 10))
        dR = np.zeros((10, 10))
        dU1 = np.zeros((10, 10))
        for k in xrange(mask):
            dU[k] = I1[k] * dHA1 * dXH1
            dR[k] = H2[k] * dHA1 * dXH1 
        Ut = U
        Rt = R
        U += alpha *((1 - sigmond(X))* dU - lamda * Ut)
        R += alpha *((1 - sigmond(X))* dR - lamda * Rt)
        #back-propagation --second
        dXH2 = np.dot(dHA1*dXH1, R.T)
        for k in xrange(mask):
            dU[k] = I2[k] * dHA2 * dXH2
            dR[k] = H3[k] * dHA2 * dXH2
        Ut = U
        Rt = R
        U += alpha *((1 - sigmond(X))* dU - lamda * Ut)
        R += alpha *((1 - sigmond(X))* dR - lamda * Rt)
        
        #    I2   I1 Inow Inext
        #    J2   J1 Jnow Jnext
        #  dHA2 dHA1 
        #  dXH2 dXH1
        #H3  H2   H1 Hnow
        
        # the forth and other items
        item = 3
        Hnow = np.zeros((1, 10))
        Hnow = Hnow[0]
        while item < isum:
            #feed-forward
            i = training_set[user][item]-1
            j = opt_set[user][item]-1
            Inext = I[i]
            Jnext = I[j]
            Anow = np.dot(Inow, U) + np.dot(Jnow, R)
            for k in xrange(mask):
                Hnow[k] = sigmond(Anow[k])
            X = np.dot(Hnow, Inext.T) - np.dot(Hnow, Jnext.T)
            dXInext = Hnow 
            dXJnext = -Hnow
            I[i] += alpha *((1 - sigmond(X))* dXInext - lamda * Inext)
            I[j] += alpha *((1 - sigmond(X))* dXJnext - lamda * Jnext)
            Inext = I[i]
            Jnext = I[j]
            #back-propagation --first
            dXHnow = Inext - Jnext
            dHAnow = np.zeros((1, 10))
            for k in xrange(mask):
                dHAnow[0][k] = Hnow[k] *(1 - Hnow[k])
            dU = np.zeros((10, 10))
            dR = np.zeros((10, 10))
            for k in xrange(mask):
                dU[k] = Inow[k] * dHAnow * dXHnow
                dR[k] = H1[k] * dHAnow * dXHnow
            Ut = U
            Rt = R
            U += alpha *((1 - sigmond(X))* dU - lamda * Ut)
            R += alpha *((1 - sigmond(X))* dR - lamda * Rt)
            
            #back-propagation --second
            dXH1 = np.dot(dHAnow*dXHnow, R.T)
            for k in xrange(mask):
                dU[k] = I1[k] * dHA1 * dXH1 ###TOHAVEWRONG!!!!!
                dR[k] = H2[k] * dHA1 * dXH1
            Ut = U
            Rt = R
            U += alpha *((1 - sigmond(X))* dU - lamda * Ut)
            R += alpha *((1 - sigmond(X))* dR - lamda * Rt)
            
            #back-propagation --third
            dXH2 = np.dot(dHA1*dXH1, R.T)
            for k in xrange(mask):
                dU[k] = I2[k] * dHA2 * dXH2
                dR[k] = H3[k] * dHA2 * dXH2
            Ut = U
            Rt = R
            U += alpha *((1 - sigmond(X))* dU - lamda * Ut)
            R += alpha *((1 - sigmond(X))* dR - lamda * Rt)           
            
            #The next step
            H3 = H2
            H2 = H1
            H1 = Hnow
            #Hnow and Hnext will be calculate at first
            I2 = I1 ; J2 = J1
            I1 = Inow ; J1 = Jnow
            Inow = Inext ; Jnow = Jnext 
            #I/Jnext will be found at first
            dHA2 = dHA1
            dHA1 = dHAnow
            item += 1
        H[user] = Hnow
train()        

In [ ]:
def predict():
    TP = 0
    P = 0
    user = 0
    while user < unum:
        H0 = np.zeros((1, 10))
        H0[0] = H[user]
        isum = len(data[user])
        if isum > 1:
            item = len(training_set[user])
            while item < isum - 1:
                temp = np.dot(H0, I.T)
                sort_list = np.argsort(-temp[0])
                for k in xrange(mask): #recall 10
                    if (sort_list[k]+1) == data[user][item+1]:
                        TP += 1
                P += 1
                It = I[data[user][item+1]-1]
                At = np.dot(H0,R) + np.dot(It,U)
                Ht = np.zeros((1, 10))
                for k in xrange(mask):
                    Ht[0][k] = sigmond(At[0][k])
                H0 = Ht
                item += 1
        user += 1
    return TP, P

In [24]:
tot_TP  = 0
tot_P = 0
alpha = 0.01
for i in xrange(100):
    print "iter", i, time.ctime()
    train()
    TP , P = predict()
    result = float(TP) / P
    print TP, P, result
    tot_TP += TP
    tot_P += P
print tot_TP/tot_P

iter 0 Sat Oct  8 00:12:23 2016


KeyboardInterrupt: 

In [ ]:
iter 0 Fri Oct  7 15:19:19 2016
7692 30117 0.255403924694
iter 1 Fri Oct  7 15:20:20 2016
7967 30117 0.264534980244
iter 2 Fri Oct  7 15:21:20 2016
7311 30117 0.242753262277
iter 3 Fri Oct  7 15:22:49 2016
7684 30117 0.255138293987
iter 4 Fri Oct  7 15:23:58 2016
7996 30117 0.265497891556
iter 5 Fri Oct  7 15:24:55 2016
8139 30117 0.270246040442
iter 6 Fri Oct  7 15:25:57 2016
7642 30117 0.253743732776
iter 7 Fri Oct  7 15:26:55 2016
7950 30117 0.263970514992
iter 8 Fri Oct  7 15:27:52 2016
7692 30117 0.255403924694
iter 9 Fri Oct  7 15:28:49 2016
7437 30117 0.246936945911
iter 10 Fri Oct  7 15:29:46 2016
7979 30117 0.264933426304
iter 11 Fri Oct  7 15:30:43 2016
7825 30117 0.259820035196
iter 12 Fri Oct  7 15:31:40 2016
7333 30117 0.243483746721
iter 13 Fri Oct  7 15:32:36 2016
8024 30117 0.26642759903
iter 14 Fri Oct  7 15:33:32 2016
7610 30117 0.252681209948
iter 15 Fri Oct  7 15:34:28 2016
7406 30117 0.245907626922
iter 16 Fri Oct  7 15:35:24 2016
7649 30117 0.253976159644
iter 17 Fri Oct  7 15:36:21 2016
7678 30117 0.254939070957
iter 18 Fri Oct  7 15:37:16 2016
8087 30117 0.268519440847
iter 19 Fri Oct  7 15:38:13 2016
7938 30117 0.263572068931
iter 20 Fri Oct  7 15:39:09 2016
7758 30117 0.257595378026
iter 21 Fri Oct  7 15:40:05 2016
7965 30117 0.264468572567
iter 22 Fri Oct  7 15:41:01 2016
7906 30117 0.262509546104
iter 23 Fri Oct  7 15:41:57 2016
7806 30117 0.259189162267
iter 24 Fri Oct  7 15:42:54 2016
7811 30117 0.259355181459
iter 25 Fri Oct  7 15:43:51 2016
7790 30117 0.258657900853
iter 26 Fri Oct  7 15:44:48 2016
7377 30117 0.244944715609
iter 27 Fri Oct  7 15:45:44 2016
7738 30117 0.256931301258
iter 28 Fri Oct  7 15:46:41 2016
7383 30117 0.245143938639
iter 29 Fri Oct  7 15:47:38 2016
7233 30117 0.240163362885
iter 30 Fri Oct  7 15:48:41 2016
7841 30117 0.26035129661
iter 31 Fri Oct  7 15:49:38 2016
7929 30117 0.263273234386
iter 32 Fri Oct  7 15:50:35 2016
7734 30117 0.256798485905
iter 33 Fri Oct  7 15:51:32 2016
7422 30117 0.246438888335
iter 34 Fri Oct  7 15:52:29 2016
8043 30117 0.267058471959
iter 35 Fri Oct  7 15:53:25 2016
7788 30117 0.258591493177
iter 36 Fri Oct  7 15:54:22 2016
7975 30117 0.264800610951
iter 37 Fri Oct  7 15:55:17 2016
7111 30117 0.236112494604
iter 38 Fri Oct  7 15:56:13 2016
7872 30117 0.261380615599
iter 39 Fri Oct  7 15:57:10 2016
8041 30117 0.266992064283
iter 40 Fri Oct  7 15:58:06 2016
8353 30117 0.277351661852
iter 41 Fri Oct  7 15:59:02 2016
7168 30117 0.238005113391
iter 42 Fri Oct  7 15:59:59 2016
7890 30117 0.26197828469
iter 43 Fri Oct  7 16:00:55 2016
7859 30117 0.2609489657
iter 44 Fri Oct  7 16:01:50 2016
7797 30117 0.258890327722
iter 45 Fri Oct  7 16:02:47 2016
7311 30117 0.242753262277
iter 46 Fri Oct  7 16:03:43 2016
8242 30117 0.273666035794
iter 47 Fri Oct  7 16:04:39 2016
7849 30117 0.260616927317
iter 48 Fri Oct  7 16:05:35 2016
8223 30117 0.273035162865
iter 49 Fri Oct  7 16:06:31 2016
8112 30117 0.269349536806
iter 50 Fri Oct  7 16:07:27 2016
7586 30117 0.251884317827
iter 51 Fri Oct  7 16:08:23 2016
7813 30117 0.259421589136
iter 52 Fri Oct  7 16:09:20 2016
7654 30117 0.254142178836
iter 53 Fri Oct  7 16:10:15 2016
7993 30117 0.265398280041
iter 54 Fri Oct  7 16:11:12 2016
7528 30117 0.249958495202
iter 55 Fri Oct  7 16:12:09 2016
7990 30117 0.265298668526
iter 56 Fri Oct  7 16:13:06 2016
8357 30117 0.277484477206
iter 57 Fri Oct  7 16:14:03 2016
7818 30117 0.259587608328
iter 58 Fri Oct  7 16:15:00 2016
7905 30117 0.262476342265
iter 59 Fri Oct  7 16:15:57 2016
8761 30117 0.290898827905
iter 60 Fri Oct  7 16:16:53 2016
8241 30117 0.273632831955
iter 61 Fri Oct  7 16:17:51 2016
8102 30117 0.269017498423
iter 62 Fri Oct  7 16:18:49 2016
8062 30117 0.267689344888
iter 63 Fri Oct  7 16:19:47 2016
7942 30117 0.263704884285
iter 64 Fri Oct  7 16:20:44 2016
7636 30117 0.253544509745
iter 65 Fri Oct  7 16:21:41 2016
7661 30117 0.254374605704
iter 66 Fri Oct  7 16:22:37 2016
7717 30117 0.256234020653
iter 67 Fri Oct  7 16:23:34 2016
8315 30117 0.276089915994
iter 68 Fri Oct  7 16:24:31 2016
7985 30117 0.265132649334
iter 69 Fri Oct  7 16:25:27 2016
7991 30117 0.265331872364
iter 70 Fri Oct  7 16:26:24 2016
7953 30117 0.264070126507
iter 71 Fri Oct  7 16:27:20 2016
7286 30117 0.241923166318
iter 72 Fri Oct  7 16:28:16 2016
7885 30117 0.261812265498
iter 73 Fri Oct  7 16:29:18 2016
7759 30117 0.257628581864
iter 74 Fri Oct  7 16:30:27 2016
7856 30117 0.260849354185
iter 75 Sat Oct  8 00:11:29 2016

In [ ]:
iter 0 Thu Oct  6 07:13:15 2016
7576 30117 0.251552279444
iter 1 Thu Oct  6 07:14:23 2016
7095 30117 0.235581233191
iter 2 Thu Oct  6 07:15:22 2016
7090 30117 0.235415213999
iter 3 Thu Oct  6 07:16:20 2016
7560 30117 0.25102101803
iter 4 Thu Oct  6 07:17:18 2016
7414 30117 0.246173257629
iter 5 Thu Oct  6 07:18:18 2016
7429 30117 0.246671315204
iter 6 Thu Oct  6 07:19:22 2016
7665 30117 0.254507421058
iter 7 Thu Oct  6 07:20:23 2016
7710 30117 0.256001593784
iter 8 Thu Oct  6 07:21:21 2016
7217 30117 0.239632101471
iter 9 Thu Oct  6 07:22:21 2016
7248 30117 0.24066142046
iter 10 Thu Oct  6 07:23:21 2016
7169 30117 0.238038317229
iter 11 Thu Oct  6 07:24:21 2016
7409 30117 0.246007238437
iter 12 Thu Oct  6 07:25:21 2016
7252 30117 0.240794235814
iter 13 Thu Oct  6 07:26:21 2016
6825 30117 0.226616196832
iter 14 Thu Oct  6 07:27:21 2016
7120 30117 0.23641132915
iter 15 Thu Oct  6 07:28:21 2016
7266 30117 0.241259089551
iter 16 Thu Oct  6 07:29:20 2016
7480 30117 0.248364710961
iter 17 Thu Oct  6 07:30:18 2016
7236 30117 0.2402629744
iter 18 Thu Oct  6 07:31:17 2016
7420 30117 0.246372480659
iter 19 Thu Oct  6 07:32:16 2016
7625 30117 0.253179267523
iter 20 Thu Oct  6 07:33:14 2016
7486 30117 0.248563933991
iter 21 Thu Oct  6 07:34:14 2016
7537 30117 0.250257329747
iter 22 Thu Oct  6 07:35:19 2016
7388 30117 0.245309957831
iter 23 Thu Oct  6 07:36:22 2016
7767 30117 0.257894212571
iter 24 Thu Oct  6 07:37:21 2016
8011 30117 0.265995949132
iter 25 Thu Oct  6 07:38:18 2016
7055 30117 0.234253079656
iter 26 Thu Oct  6 07:39:17 2016
7182 30117 0.238469967128
iter 27 Thu Oct  6 07:40:16 2016
7358 30117 0.24431384268
iter 28 Thu Oct  6 07:41:14 2016
7400 30117 0.245708403891
iter 29 Thu Oct  6 07:42:13 2016
7411 30117 0.246073646113
iter 30 Thu Oct  6 07:43:12 2016
7389 30117 0.245343161669
iter 31 Thu Oct  6 07:44:12 2016
7337 30117 0.243616562075
iter 32 Thu Oct  6 07:45:13 2016
7449 30117 0.247335391971
iter 33 Thu Oct  6 07:46:15 2016
7292 30117 0.242122389348
iter 34 Thu Oct  6 07:47:18 2016
7138 30117 0.23700899824
iter 35 Thu Oct  6 07:48:20 2016
7477 30117 0.248265099445
iter 36 Thu Oct  6 07:49:24 2016
7540 30117 0.250356941262
iter 37 Thu Oct  6 07:50:28 2016
7732 30117 0.256732078228
iter 38 Thu Oct  6 07:51:29 2016
7278 30117 0.241657535611
iter 39 Thu Oct  6 07:52:33 2016
7702 30117 0.255735963077
iter 40 Thu Oct  6 07:53:35 2016
7596 30117 0.252216356211
iter 41 Thu Oct  6 07:54:36 2016
7561 30117 0.251054221868
iter 42 Thu Oct  6 07:55:40 2016
7392 30117 0.245442773185
iter 43 Thu Oct  6 07:56:40 2016
7509 30117 0.249327622273
iter 44 Thu Oct  6 07:57:41 2016
7590 30117 0.252017133181
iter 45 Thu Oct  6 07:58:42 2016
7387 30117 0.245276753993
iter 46 Thu Oct  6 07:59:42 2016
7562 30117 0.251087425706
iter 47 Thu Oct  6 08:00:45 2016
7594 30117 0.252149948534
iter 48 Thu Oct  6 08:01:47 2016
7255 30117 0.240893847329
iter 49 Thu Oct  6 08:02:48 2016
7709 30117 0.255968389946
iter 50 Thu Oct  6 08:03:50 2016
7384 30117 0.245177142478
iter 51 Thu Oct  6 08:04:52 2016
7357 30117 0.244280638842
iter 52 Thu Oct  6 08:05:54 2016
7468 30117 0.2479662649
iter 53 Thu Oct  6 08:06:55 2016
7556 30117 0.250888202676
iter 54 Thu Oct  6 08:07:56 2016
7465 30117 0.247866653385
iter 55 Thu Oct  6 08:08:57 2016
7915 30117 0.262808380649
iter 56 Thu Oct  6 08:09:58 2016
7769 30117 0.257960620248
iter 57 Thu Oct  6 08:11:00 2016
7872 30117 0.261380615599
iter 58 Thu Oct  6 08:11:58 2016
7937 30117 0.263538865093
iter 59 Thu Oct  6 08:12:55 2016
7396 30117 0.245575588538
iter 60 Thu Oct  6 08:13:53 2016
7642 30117 0.253743732776
iter 61 Thu Oct  6 08:14:51 2016
7731 30117 0.25669887439
iter 62 Thu Oct  6 08:15:48 2016
7188 30117 0.238669190158
iter 63 Thu Oct  6 08:16:46 2016
7742 30117 0.257064116612
iter 64 Thu Oct  6 08:17:44 2016
8054 30117 0.267423714181
iter 65 Thu Oct  6 08:18:41 2016
7885 30117 0.261812265498
iter 66 Thu Oct  6 08:19:39 2016
7915 30117 0.262808380649
iter 67 Thu Oct  6 08:20:36 2016
7993 30117 0.265398280041
iter 68 Thu Oct  6 08:21:35 2016
7921 30117 0.263007603679
iter 69 Thu Oct  6 08:22:32 2016
8001 30117 0.265663910748
iter 70 Thu Oct  6 08:23:30 2016
8076 30117 0.268154198625
iter 71 Thu Oct  6 08:24:27 2016
8000 30117 0.26563070691
iter 72 Thu Oct  6 08:25:25 2016
7452 30117 0.247435003486
iter 73 Thu Oct  6 08:26:23 2016

iter 0 Thu Oct  6 00:06:37 2016
6415 30117 0.213002623103
iter 1 Thu Oct  6 00:10:21 2016
6594 30117 0.21894611017
iter 2 Thu Oct  6 00:14:11 2016
6076 30117 0.201746521898
iter 3 Thu Oct  6 00:17:56 2016
6350 30117 0.21084437361
iter 4 Thu Oct  6 00:21:45 2016
6647 30117 0.220705913604
iter 5 Thu Oct  6 00:25:29 2016
6142 30117 0.20393797523
iter 6 Thu Oct  6 00:29:08 2016
6613 30117 0.219576983099
iter 7 Thu Oct  6 00:32:14 2016
6569 30117 0.218116014211
iter 8 Thu Oct  6 00:34:46 2016
6320 30117 0.209848258459
iter 9 Thu Oct  6 00:37:23 2016
6637 30117 0.22037387522
iter 10 Thu Oct  6 00:40:01 2016
6645 30117 0.220639505927
iter 11 Thu Oct  6 00:42:36 2016
6470 30117 0.214828834213
iter 12 Thu Oct  6 00:45:14 2016
6621 30117 0.219842613806
iter 13 Thu Oct  6 00:47:45 2016
6638 30117 0.220407079058
iter 14 Thu Oct  6 00:50:14 2016
6498 30117 0.215758541687
iter 15 Thu Oct  6 00:52:44 2016
6580 30117 0.218481256433
iter 16 Thu Oct  6 00:55:12 2016
6197 30117 0.20576418634
iter 17 Thu Oct  6 00:57:41 2016
6524 30117 0.216621841485
iter 18 Thu Oct  6 01:00:13 2016
6414 30117 0.212969419265
iter 19 Thu Oct  6 01:02:46 2016
6458 30117 0.214430388153
iter 20 Thu Oct  6 01:05:18 2016
6784 30117 0.225254839459
iter 21 Thu Oct  6 01:07:53 2016
6586 30117 0.218680479463
iter 22 Thu Oct  6 01:10:29 2016
6800 30117 0.225786100873
iter 23 Thu Oct  6 01:13:04 2016
6413 30117 0.212936215427
iter 24 Thu Oct  6 01:15:37 2016
6565 30117 0.217983198858
iter 25 Thu Oct  6 01:18:12 2016
6589 30117 0.218780090979
iter 26 Thu Oct  6 01:20:41 2016
6611 30117 0.219510575423
iter 27 Thu Oct  6 01:23:15 2016
6845 30117 0.2272802736
iter 28 Thu Oct  6 01:25:51 2016
6565 30117 0.217983198858
iter 29 Thu Oct  6 01:28:07 2016
6951 30117 0.230799880466
iter 30 Thu Oct  6 01:30:09 2016
7090 30117 0.235415213999
iter 31 Thu Oct  6 01:32:11 2016
6824 30117 0.226582992994
iter 32 Thu Oct  6 01:34:11 2016
6609 30117 0.219444167746
iter 33 Thu Oct  6 01:36:11 2016
6816 30117 0.226317362287
iter 34 Thu Oct  6 01:38:12 2016
6677 30117 0.221702028755
iter 35 Thu Oct  6 01:40:13 2016
6968 30117 0.231364345718
iter 36 Thu Oct  6 01:42:14 2016
6855 30117 0.227612311983
iter 37 Thu Oct  6 01:44:14 2016
6724 30117 0.223262609158
iter 38 Thu Oct  6 01:46:15 2016
7382 30117 0.245110734801
iter 39 Thu Oct  6 01:48:16 2016
6851 30117 0.22747949663
iter 40 Thu Oct  6 01:50:17 2016
6965 30117 0.231264734203
iter 41 Thu Oct  6 01:52:18 2016
6920 30117 0.229770561477
iter 42 Thu Oct  6 01:54:19 2016
6981 30117 0.231795995617
iter 43 Thu Oct  6 01:56:19 2016
6901 30117 0.229139688548
iter 44 Thu Oct  6 01:58:20 2016
6630 30117 0.220141448351
iter 45 Thu Oct  6 02:00:19 2016
7177 30117 0.238303947936
iter 46 Thu Oct  6 02:01:19 2016
7050 30117 0.234087060464
iter 47 Thu Oct  6 02:02:17 2016
7131 30117 0.236776571372
iter 48 Thu Oct  6 02:03:16 2016
6867 30117 0.228010758044
iter 49 Thu Oct  6 02:04:14 2016
7178 30117 0.238337151775
iter 50 Thu Oct  6 02:05:13 2016
6726 30117 0.223329016834
iter 51 Thu Oct  6 02:06:12 2016
6989 30117 0.232061626324
iter 52 Thu Oct  6 02:07:10 2016
6951 30117 0.230799880466
iter 53 Thu Oct  6 02:08:09 2016
7085 30117 0.235249194807
iter 54 Thu Oct  6 02:09:08 2016
7056 30117 0.234286283494
iter 55 Thu Oct  6 02:10:07 2016
7134 30117 0.236876182887
iter 56 Thu Oct  6 02:11:06 2016
6876 30117 0.228309592589
iter 57 Thu Oct  6 02:12:05 2016
6865 30117 0.227944350367
iter 58 Thu Oct  6 02:13:04 2016
7120 30117 0.23641132915
iter 59 Thu Oct  6 02:14:03 2016
6491 30117 0.215526114819
iter 60 Thu Oct  6 02:15:03 2016
6837 30117 0.227014642893
iter 61 Thu Oct  6 02:16:04 2016
7142 30117 0.237141813594
iter 62 Thu Oct  6 02:17:05 2016
6801 30117 0.225819304712
iter 63 Thu Oct  6 02:18:04 2016
6708 30117 0.222731347744
iter 64 Thu Oct  6 02:19:03 2016
7085 30117 0.235249194807
iter 65 Thu Oct  6 02:20:02 2016
6869 30117 0.22807716572
iter 66 Thu Oct  6 02:21:01 2016
6906 30117 0.22930570774
iter 67 Thu Oct  6 02:22:00 2016
7039 30117 0.233721818242
iter 68 Thu Oct  6 02:22:58 2016
7110 30117 0.236079290766
iter 69 Thu Oct  6 02:23:57 2016
6964 30117 0.231231530365
iter 70 Thu Oct  6 02:24:57 2016
6849 30117 0.227413088953
iter 71 Thu Oct  6 02:25:57 2016
7184 30117 0.238536374805
iter 72 Thu Oct  6 02:26:57 2016
7090 30117 0.235415213999
iter 73 Thu Oct  6 02:27:56 2016
7050 30117 0.234087060464
iter 74 Thu Oct  6 02:28:57 2016
6972 30117 0.231497161072
iter 75 Thu Oct  6 02:29:57 2016
7018 30117 0.233024537637
iter 76 Thu Oct  6 02:30:57 2016
7025 30117 0.233256964505
iter 77 Thu Oct  6 02:31:57 2016
7142 30117 0.237141813594
iter 78 Thu Oct  6 02:32:57 2016
7300 30117 0.242388020055
iter 79 Thu Oct  6 02:33:57 2016
7019 30117 0.233057741475
iter 80 Thu Oct  6 02:34:57 2016
7007 30117 0.232659295415
iter 81 Thu Oct  6 02:35:57 2016
7165 30117 0.237905501876
iter 82 Thu Oct  6 02:36:57 2016
7013 30117 0.232858518445
iter 83 Thu Oct  6 02:37:57 2016
6827 30117 0.226682604509
iter 84 Thu Oct  6 02:38:57 2016
7494 30117 0.248829564698
iter 85 Thu Oct  6 02:39:58 2016
7409 30117 0.246007238437
iter 86 Thu Oct  6 02:40:58 2016
7181 30117 0.23843676329
iter 87 Thu Oct  6 02:41:57 2016
7636 30117 0.253544509745
iter 88 Thu Oct  6 02:42:56 2016
7168 30117 0.238005113391
iter 89 Thu Oct  6 02:43:55 2016
7543 30117 0.250456552778
iter 90 Thu Oct  6 02:44:54 2016
7604 30117 0.252481986918
iter 91 Thu Oct  6 02:45:53 2016
7281 30117 0.241757147126
iter 92 Thu Oct  6 02:46:53 2016
7312 30117 0.242786466115
iter 93 Thu Oct  6 02:47:52 2016
7178 30117 0.238337151775
iter 94 Thu Oct  6 02:48:52 2016
7099 30117 0.235714048544
iter 95 Thu Oct  6 02:49:52 2016
7227 30117 0.239964139855
iter 96 Thu Oct  6 02:50:51 2016
7261 30117 0.241093070359
iter 97 Thu Oct  6 02:51:51 2016
7604 30117 0.252481986918
iter 98 Thu Oct  6 02:52:51 2016
7604 30117 0.252481986918
iter 99 Thu Oct  6 02:53:50 2016
7289 30117 0.242022777833
0

#Modefied
The new copy reset the opt_set in every training set
and use a new learning data : 0.05
The iter 0 in fact is iter 7

iter 15 Thu Oct  6 00:01:53 2016
4199 30117 0.139422917289
changed alpha = 0.01
